In [1]:
import numpy as np
import pandas as pd
import time

from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, \
                            auc, \
                            confusion_matrix, \
                            log_loss, make_scorer, \
                            roc_auc_score, roc_curve, \
                            precision_recall_curve, \
                            precision_score, \
                            recall_score, \
                            f1_score
from sklearn.model_selection import GridSearchCV, \
                                    train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler

In [2]:
df = pd.read_pickle('../data/preprocessed_training_data.pkl')

In [3]:
def log_loss_score(clf, x, y):
    return log_loss(y, clf.predict_proba(x))

def auroc_score(clf, x, y):
    return roc_auc_score(y, clf.predict_proba(x)[:, 1])

In [4]:
numerical_columns = ['startCount', 'viewCount', 'installCount', 'startCount1d', 'startCount7d', 'timeSinceLastStart']
categorical_columns = ['campaignId', 'sourceGameId', 'country']

numerical_pipeline = make_pipeline(RobustScaler())
categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown='ignore'))

preprocessor = ColumnTransformer(
    [('numerical_preprocessing', numerical_pipeline, numerical_columns), 
     ('categorical_preprocessing', categorical_pipeline, categorical_columns)], 
    remainder='drop')

In [5]:
X = df[numerical_columns + categorical_columns]
y = df['install']

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [18]:
pipeline = make_pipeline(preprocessor, 
                         RandomUnderSampler(random_state=0), 
                         SGDClassifier(loss='hinge', random_state=0))

In [19]:
param_range = [0.00001, 0.0001, 0.001]
param_grid = [{'sgdclassifier__alpha': param_range, 
               'sgdclassifier__penalty': ['l1', 'l2']}]

In [20]:
t_0 = time.time()
gs = GridSearchCV(estimator=pipeline,
                  param_grid=param_grid,
                  scoring='roc_auc',
                  cv=3)
gs.fit(X_train, y_train)
print('{} minutes'.format((time.time() - t_0) / 60.0))
print(gs.best_score_)
print(gs.best_params_)

4.3116312225659685 minutes
0.7151309730514122
{'sgdclassifier__alpha': 0.0001, 'sgdclassifier__penalty': 'l2'}


The hyperparameter tunining took approximately 7 minutes with the optimal penalty l1 and alpha value of 0.0001.

## Optimal Classifier Training

In [23]:
t_0 = time.time()
pipeline = make_pipeline(preprocessor, 
                         RandomUnderSampler(random_state=0), 
                         SGDClassifier(loss='log', penalty='l1', alpha=0.0001, random_state=0, max_iter=2000))
pipeline.fit(X_train, y_train)
print('{} seconds'.format((time.time() - t_0)))

12.719181299209595 seconds


In [24]:
y_pred = pipeline.predict(X_test)
print("Precision: {}%".format(int(100 * precision_score(y_test, y_pred))))
print("Recall: {}%".format(int(100 * recall_score(y_test, y_pred))))
print("Log-loss: {}%".format(int(100 * log_loss_score(pipeline, X_test, y_test))))
print("AUROC: {}%".format(int(100 * roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))))
tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()
print("True Negatives: {}, Fale Positives: {}, False Negatives: {}, True Positives: {}".format(tn, fp, fn, tp))
print("Prediction bias: {}".format(sum(y_pred) / len(y_pred) - sum(y_test) / len(y_test)))

Precision: 2%
Recall: 52%
Log-loss: 60%
AUROC: 69%
True Negatives: 536021, Fale Positives: 4213, False Negatives: 202818, True Positives: 4736
Prediction bias: 0.26558998004782103
